In [1]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
mixer.init()
sound = mixer.Sound('warning-alarm.WAV')
face = cv2.CascadeClassifier('HAARCASCADEfiles\haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('HAARCASCADEfiles\haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('HAARCASCADEfiles\haarcascade_righteye_2splits.xml')
lbl=['Close','Open']

In [3]:
model = load_model('my_model.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]

In [ ]:
use_webcam = False  # Set to False to stop webcam check

if use_webcam:
    cap = cv2.VideoCapture(0)  # Make sure to initialize cap here
    while True:
        ret, frame = cap.read()
        if not ret or frame is None:
            print("Failed to grab frame. Exiting loop.")
            break

        height, width = frame.shape[:2]
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face.detectMultiScale(gray, minNeighbors=5, scaleFactor=1.1, minSize=(25, 25))
        left_eye = leye.detectMultiScale(gray)
        right_eye = reye.detectMultiScale(gray)

        cv2.rectangle(frame, (0, height-50), (200, height), (0, 0, 0), thickness=cv2.FILLED)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 5)

        for (x, y, w, h) in right_eye:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            r_eye = frame[y:y+h, x:x+w]
            count = count + 1
            r_eye = cv2.cvtColor(r_eye, cv2.COLOR_BGR2GRAY)
            r_eye = cv2.resize(r_eye, (48, 48))
            r_eye = r_eye / 255
            r_eye = r_eye.reshape(48, 48, -1)
            r_eye = np.expand_dims(r_eye, axis=0)
            predict_x = model.predict(r_eye)
            rpred = np.argmax(predict_x, axis=1)
            if rpred[0] == 1:
                lbl = 'Open'
            if rpred[0] == 0:
                lbl = 'Closed'
            break

        for (x, y, w, h) in left_eye:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            l_eye = frame[y:y+h, x:x+w]
            count = count + 1
            l_eye = cv2.cvtColor(l_eye, cv2.COLOR_BGR2GRAY)
            l_eye = cv2.resize(l_eye, (48, 48))
            l_eye = l_eye / 255
            l_eye = l_eye.reshape(48, 48, -1)
            l_eye = np.expand_dims(l_eye, axis=0)
            predict_x = model.predict(l_eye)
            lpred = np.argmax(predict_x, axis=1)
            if lpred[0] == 1:
                lbl = 'Open'
            if lpred[0] == 0:
                lbl = 'Closed'
            break

        if (rpred[0] == 0 and lpred[0] == 0):
            score = score + 1
            cv2.putText(frame, "Closed: You are feeling drowsy!! ALERT!!", (10, height-20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        else:
            score = score - 1
            cv2.putText(frame, "Open", (10, height-20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)

        if score < 0:
            score = 0

        if score > 5:
            # person is feeling sleepy so we beep the alarm
            cv2.imwrite(os.path.join(path, 'image.jpg'), frame)
            try:
                sound.play()
            except:
                pass
            if thicc < 14:
                thicc = thicc + 2
            else:
                thicc = thicc - 2
                if thicc < 2:
                    thicc = 2
            cv2.rectangle(frame, (0, 0), (width, height), (0, 0, 255), thicc)

        cv2.imshow('frame', frame)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
else:
    print("Webcam check is disabled.")

1/1 [==============================] - 0s 34ms/step


KeyboardInterrupt: 